# Socket Communication Network

This notebook is an implementation of a simple UDP socket server using *CoppeliaSim's* threaded childscripts as clients. 

The goal is to implement and analyse, the characteristics of the User Datagram Protocol for communication between the client and the server. For this:
1. Associate threaded childscripts to *CoppeliaSim's* Dummies;
2. Code the client side on these childscripts;
3. Send messages each sensing callback is done to the server;
4. Check Real Time Factor, Latency and Sending Rate. 

---

In [16]:
# Importing modules...
import numpy as np
import socket		

import sys
sys.path.append('../..') # Go back to base directory

from modules.plot.graph import Graph

# Creating the Server's UDP Socket

The information bellow is taken from the GeeksForGeeks's website and is available [here](https://www.geeksforgeeks.org/differences-between-tcp-and-udp/).

## The User Datagram Protocol

User Datagram Protocol (UDP) is a Transport Layer Protocol. UDP is a part of the Internet Protocol suite, referred to as the UDP/IP suite. Unlike the Transmission Control Protocol (TCP), it is an unreliable and connectionless protocol. 

Therefore, there is no need to establish a connection before data transfer. The UDP helps to establish **low-latency** and **loss-tolerating** connections establish over the network. The UDP enables process-to-process communication - an event based communication.

- Used for simple request-response communication when the size of data is less and hence there is lesser concern about flow and error control.
- It is a suitable protocol for multicasting as UDP supports packet switching.
- UDP is used for some routing update protocols like RIP (Routing Information Protocol).
- Normally used for real-time applications which can not tolerate uneven delays between sections of a received message.

### Advantages of UDP
- It does not require any connection for sending or receiving data;
- Broadcast and Multicast are available in UDP;
- UDP can operate on a large range of networks;
- UDP has live and real-time data;
- UDP can deliver data if all the components of the data are not complete.

### Disadvantages of UDP
- We can not have any way to acknowledge the successful transfer of data;
- UDP cannot have the mechanism to track the sequence of data;
- UDP is connectionless, and due to this, it is unreliable to transfer data;
- In case of a Collision, UDP packets are dropped by routers in comparison to TCP;
- UDP can drop packets in case of detection of errors.

---

In [17]:
print('[SERVER] Creating socket...')

# Try to create server socket
try: 
    server_socket = socket.socket(socket.AF_INET,    # Internet
                                  socket.SOCK_DGRAM) # UDP
    print('[SERVER] Socket successfully created')
    
except socket.error as err: 
    print(f'[SERVER] Socket creation failed with error {err}\n')
    print('> Quitting code...')
    exit()

server_ip = '127.0.0.1' # Server IP
server_port = 8888      # Server Port
server_address = (server_ip, server_port) 

server_socket.bind(server_address)
print(f'[SERVER] Bound to port {server_port}')

buffer_size = 1024 # Size of the messages in bytes

[SERVER] Creating socket...
[SERVER] Socket successfully created
[SERVER] Bound to port 8888


# Identifying Clients Addresses

After starting the server, it will wait for clients to send messages to it with their ID number and save their addresses (IP and Port). This routine waits until all clients identify themselves. 

---

In [18]:
n_clients = 4
client_addresses = {}

print('[SERVER] Waiting for clients...')

# Address lookup 
while len(client_addresses.keys()) < n_clients: # Until all clients are identified
    message_bytes, address = server_socket.recvfrom(buffer_size)

    try:
        ID = int(message_bytes.decode()) # Decode message

    except: # Invalid message for decoding
        continue # Look for another message

    client_addresses[address] = ID

    print(f'\tClient {ID} Connected')

print('[SERVER] All clients connected!')

[SERVER] Waiting for clients...
	Client 0 Connected
	Client 1 Connected
	Client 2 Connected
	Client 3 Connected
[SERVER] All clients connected!


# Receiving Client Messages

With all the clients identified, the server will now start receiving the desired messages sent by them. For avoiding the server to run indefinetely, a timeout will be set to close the socket after the timeout. 

In this simplified case, clients will only send the Simulation Timestamp they are being triggered to send the message.  

---

In [19]:
verbose = True

timeout = 5 # In seconds
server_socket.settimeout(timeout) # Set server timeout
print(f'[SERVER] Timeout set to {timeout} seconds\n')

# Saving messaging profile
received_PTS = [[] for _ in range(n_clients)]

while True:  
    # Wait for message - Event guided!
    try:
        message_bytes, address = server_socket.recvfrom(buffer_size)

    except socket.timeout as err:
        print('\n[SERVER] Timed Out!')
        
        break # Close capture loop due to timeout
    
    # Check if client exists
    try:
        ID = client_addresses[address] # Client Identifier
    
    except:
        if verbose: print('> Client not recognized')

        continue # Jump to wait for the next message

    # Decode message
    try:
        message = np.frombuffer(message_bytes, dtype=np.float64)

    except:
        if verbose: print('> Couldn\'t decode message')

        continue # Jump to wait for the next message

    # Empty message
    if not message.size:
        if verbose: print(f'\tEmpty message')

        continue # Jump to wait for the next message
    
    # Decode message 
    [PTS] = np.frombuffer(message_bytes, dtype=np.float64)

    if verbose:
        print(f'> Received message from Client {client_addresses[address]} ({address[0]}, {address[1]}):')
        print(f'\tPTS: {PTS :.3f} s')

    # Save message
    received_PTS[ID].append(PTS)


[SERVER] Timeout set to 5 seconds

> Received message from Client 0 (127.0.0.1, 61844):
	PTS: 0.050 s
> Received message from Client 1 (127.0.0.1, 61853):
	PTS: 0.050 s
> Received message from Client 2 (127.0.0.1, 61854):
	PTS: 0.050 s
> Received message from Client 3 (127.0.0.1, 54815):
	PTS: 0.050 s
> Received message from Client 0 (127.0.0.1, 61844):
	PTS: 0.100 s
> Received message from Client 1 (127.0.0.1, 61853):
	PTS: 0.100 s
> Received message from Client 2 (127.0.0.1, 61854):
	PTS: 0.100 s
> Received message from Client 3 (127.0.0.1, 54815):
	PTS: 0.100 s
> Received message from Client 0 (127.0.0.1, 61844):
	PTS: 0.150 s
> Received message from Client 1 (127.0.0.1, 61853):
	PTS: 0.150 s
> Received message from Client 2 (127.0.0.1, 61854):
	PTS: 0.150 s
> Received message from Client 3 (127.0.0.1, 54815):
	PTS: 0.150 s
> Received message from Client 0 (127.0.0.1, 61844):
	PTS: 0.200 s
> Received message from Client 1 (127.0.0.1, 61853):
	PTS: 0.200 s
> Received message from Cli

# Plotting Messaging Profile

To analyse the behaviour of the received messages in respect to when they're being sended regarding simulation time, a graph will be plotted for visualization.

Expect some messages to not be parsed due to UDP message loss!

---

In [20]:
# Create graph
messages_plot = Graph(title='Received Messages',
                      axis_title=('PTS', 'Clients'))

# Adding received client messages
for ID in range(n_clients):
    client_messages = np.array(received_PTS[ID])
    messages_per_PTS = np.vstack((client_messages, np.full(client_messages.shape, ID)))
    messages_plot.add_points(messages_per_PTS, f'Client {ID}')

# Plot image
messages_plot.figure.show(renderer='notebook_connected')